In [204]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection
nltk.download('punkt')

from transformers import BertTokenizer

[nltk_data] Downloading package punkt to /home/moonstar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [205]:
df_train = pd.read_csv('open/train.csv', delimiter=',', encoding = "utf8")
print(len(df_train))

# NaN값 삭제
df_train.dropna(inplace=True)
print(len(df_train))

print(df_train['facts'][0])
df_train.head()

2478
2478
On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. 
Thompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court held that the First Amendment protects uninhibited, robust debate, 

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [206]:
sentences = []
first_people = []
second_people = []
winners = []


for j in range(len(df_train)):

    first_party = df_train['first_party'][j]
    second_party = df_train['second_party'][j]
    # print(first_party)
    # print(second_party)

    first_party_tokens = nltk.tokenize.word_tokenize(first_party)
    second_party_tokens = nltk.tokenize.word_tokenize(second_party)
    # print(first_party_tokens)
    # print(second_party_tokens)


    fact = df_train['facts'][j]
    fact_tokens = nltk.tokenize.word_tokenize(fact)

    first = 'tony'
    second = 'jack'

    first_people.append(first)
    # first_people.append(second)
    second_people.append(second)
    # second_people.append(first)

    win = df_train['first_party_winner'][j]
    winners.append(win)
    # winners.append((win*-1)+1)

    name_tmp = []
    new = []
    new2 = []

    new_sentence1 = ''
    new_sentence2 = ''

    for i in range(len(fact_tokens)):
        token = fact_tokens[i]
        token2 = fact_tokens[i]
        plus = True

        if token in first_party_tokens:
            if token == first_party_tokens[-1]:
                token = first 
                token2 = second

            else:
                token = ''
                plus = False 

        if token in second_party_tokens:
            if token == second_party_tokens[-1]:
                token = second 
                token2 = first 

            else:
                token = ''
                plus = False

        if plus == True:
            new.append(token)
            # new2.append(token2)

            new_sentence1 += token + ' '
            # new_sentence2 += token2 + ' '


    # print(new)
    # print(new2)

    # print(new_sentence1)
    # print(new_sentence2)

    sentences.append(new_sentence1)
    # sentences.append(new_sentence2)




In [207]:
# sentences = []
# first_people = []
# second_people = []
# winners = []

# print(sentences[:5])
print(first_people[:6])
print(second_people[:6])
print(winners[:20])

print(len(sentences))
print(len(first_people))
print(len(second_people))
print(len(winners))

['tony', 'tony', 'tony', 'tony', 'tony', 'tony']
['jack', 'jack', 'jack', 'jack', 'jack', 'jack']
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
2478
2478
2478
2478


In [208]:
first_party_data_frame = pd.DataFrame(first_people, columns=['first_party'])
second_party_data_frame = pd.DataFrame(second_people, columns=['second_party'])
facts_data_frame = pd.DataFrame(sentences, columns=['facts'])
winners_data_frame = pd.DataFrame(winners, columns=['first_party_winner'])

train_data_frame = pd.concat([first_party_data_frame, second_party_data_frame, facts_data_frame, winners_data_frame],axis=1,join='inner')
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
2473,tony,jack,Congress amended the Clean Air Act through the...,1
2474,tony,jack,Inc. an investment fund purchased approximatel...,1
2475,tony,jack,"In 1992 , the District Court sentenced Manuel ...",0
2476,tony,jack,"On March 8 , 1996 , Enrico jack , a lawful per...",0


In [209]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [210]:
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
2473,tony,jack,Congress amended the Clean Air Act through the...,1
2474,tony,jack,Inc. an investment fund purchased approximatel...,1
2475,tony,jack,"In 1992 , the District Court sentenced Manuel ...",0
2476,tony,jack,"On March 8 , 1996 , Enrico jack , a lawful per...",0


In [211]:
vectorizer = CountVectorizer()

xfeatures = train_data_frame['facts']
ylabel = train_data_frame['first_party_winner']

In [263]:
# X_train, X_test, y_train, y_test = train_test_split(xfeatures, ylabel, test_size=0.25)
X_train, X_test, y_train, y_test = train_test_split(xfeatures, ylabel, test_size=0.33, random_state=321)


# print(len(X_train))
# print(len(X_test))


X_train = xfeatures
y_train = ylabel


# k = 1500
# X_train = xfeatures[:k]
# y_train = ylabel[:k]

# X_test = xfeatures[k:]
# y_test = ylabel[k:]

print(len(X_train))
print(len(X_test))

2478
818


In [264]:
pipe = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(solver='liblinear'))])

In [265]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr', LogisticRegression(solver='liblinear'))])

In [266]:
pipe.score(X_train,y_train)

0.9995964487489911

In [267]:
pipe.score(X_test,y_test)

# y_pred = pipe.predict(X_test)
# print(y_pred)

0.9987775061124694

: 

In [253]:
"""
Test data
"""


df_test = pd.read_csv('open/test.csv', delimiter=',', encoding = "utf8")
print(len(df_test))

# NaN값 삭제
df_test.dropna(inplace=True)
print(len(df_test))

print(df_test['facts'][0])
df_test.head()

1240
1240
The 1984 Bail Reform Act allowed the federal courts to detain an arrestee prior to trial if the government could prove that the individual was potentially dangerous to other people in the community. Prosecutors alleged that Salerno and another person in this case were prominent figures in the La Cosa Nostra crime family.



,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [254]:
sentences = []
first_people = []
second_people = []


for j in range(len(df_test)):

    first_party = df_test['first_party'][j]
    second_party = df_test['second_party'][j]
    # print(first_party)
    # print(second_party)

    first_party_tokens = nltk.tokenize.word_tokenize(first_party)
    second_party_tokens = nltk.tokenize.word_tokenize(second_party)
    # print(first_party_tokens)
    # print(second_party_tokens)


    fact = df_test['facts'][j]
    fact_tokens = nltk.tokenize.word_tokenize(fact)

    first = 'tony'
    second = 'jack'

    first_people.append(first)
    # first_people.append(second)
    second_people.append(second)
    # second_people.append(first)

    name_tmp = []
    new = []
    new2 = []

    new_sentence1 = ''
    new_sentence2 = ''

    for i in range(len(fact_tokens)):
        token = fact_tokens[i]
        token2 = fact_tokens[i]
        plus = True

        if token in first_party_tokens:
            if token == first_party_tokens[-1]:
                token = first 
                token2 = second

            else:
                token = ''
                plus = False 

        if token in second_party_tokens:
            if token == second_party_tokens[-1]:
                token = second 
                token2 = first 

            else:
                token = ''
                plus = False

        if plus == True:
            new.append(token)
            # new2.append(token2)

            new_sentence1 += token + ' '
            # new_sentence2 += token2 + ' '


    # print(new)
    # print(new2)

    # print(new_sentence1)
    # print(new_sentence2)

    sentences.append(new_sentence1)
    # sentences.append(new_sentence2)


In [255]:
print(len(sentences))
print(len(first_people))
print(len(second_people))

first_party_data_frame = pd.DataFrame(first_people, columns=['first_party'])
second_party_data_frame = pd.DataFrame(second_people, columns=['second_party'])
facts_data_frame = pd.DataFrame(sentences, columns=['facts'])

test_data_frame = pd.concat([first_party_data_frame, second_party_data_frame, facts_data_frame],axis=1,join='inner')
test_data_frame

1240
1240
1240


,first_party,second_party,facts
0,tony,jack,The 1984 Bail Reform Act allowed the federal c...
1,tony,jack,Inc. was a defendant in a class action lawsuit...
2,tony,jack,In 2002 and 2003 broadcast the Billboard Music...
3,tony,jack,During his trial for armed robbery of a federa...
4,tony,jack,"In 1993 , a magistrate judge issued a warrant ..."
...,...,...,...
1235,tony,jack,According to Executive Order No tony 12807 sig...
1236,tony,jack,Section 109 ( a ) of the Clean Air Act ( CAA )...
1237,tony,jack,tony created a plan for utilizing $ 2.6 millio...
1238,tony,jack,"In 1972 , the North Carolina Board of Agricult..."


In [256]:
X_test = test_data_frame['facts']

y_pred = pipe.predict(X_test)
print(len(y_pred))
print(y_pred)

1240
[0 1 1 ... 1 1 1]


In [257]:
y_pred = pd.DataFrame(y_pred, columns=['first_party_winner'])
y_pred

,first_party_winner
0,0
1,1
2,1
3,1
4,1
...,...
1235,1
1236,0
1237,1
1238,1


In [258]:
prediction = pd.concat([df_test['ID'],y_pred],axis=1,join='inner')
prediction

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,1
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,1
...,...,...
1235,TEST_1235,1
1236,TEST_1236,0
1237,TEST_1237,1
1238,TEST_1238,1


In [259]:
prediction.to_csv('./baseline_submit.csv', index=False)
print('Done')

Done
